# Fine-tuning a model with the Trainer Wandb Chinese

## 安裝套件

In [1]:
!pip install datasets evaluate transformers[sentencepiece] --quiet
!pip install accelerate -U --quiet
!pip install wandb --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.1/281.1 kB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.3 MB/s eta 0:00:00


In [3]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: ERROR API key must be 40 characters long, yours was 63


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [4]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset("sepidmnorozy/Chinese_sentiment")
checkpoint = "google-bert/bert-base-chinese"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


def tokenize_function(example):
    return tokenizer(example["text"], truncation=True)


tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Generating train split:   0%|          | 0/12348 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2591 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/4896 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/269k [00:00<?, ?B/s]

Map:   0%|          | 0/12348 [00:00<?, ? examples/s]

Map:   0%|          | 0/2591 [00:00<?, ? examples/s]

Map:   0%|          | 0/4896 [00:00<?, ? examples/s]

### 定義`Trainer`的參數
我們定義我們之前的第一步`Trainer`是定義一個類別，其中包含將用於訓練和評估的`TrainingArguments`所有超參數。為求簡單，這邊`Trainer`提供的參數是儲存訓練模型的目錄以及訓練的輪數。對於其餘所有內容，保留預設值，這對於基本的微調應該非常有效。

In [5]:
from transformers import TrainingArguments

training_args = TrainingArguments("test-trainer", num_train_epochs=3, report_to="wandb")

使用`AutoModelForSequenceClassification`有兩個標籤的類別

In [6]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model.safetensors:   0%|          | 0.00/412M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-chinese and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### 說明
在實例化這個預訓練模型後會收到一個警告。這是因為BERT沒有在對句子對進行分類的任務上進行預訓練，因此預訓練模型的頭部被丟棄了(bert-base-uncase)，並且加入了一個適合序列分類的新頭部。表示有些權重未被使用（對應於被丟棄的預訓練頭部的那些）以及一些其他權重是隨機初始化的（新頭部）

### 定義`Trainer`
有了模型，就可以傳入建構的所有對象來定義一個訓練器模型、`training_args`、訓練和驗證資料集、`data_collator`和分詞器

In [7]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

### 說明
這樣傳遞分詞器時，訓練器使用的預設`data_collator`將是之前定義的`DataCollatorWithPadding`，因此可以跳過`data_collator=data_collator`這一行

### 開始訓練
呼叫`Trainer`中的`train()`方法

In [8]:
trainer.train()

wandb: Currently logged in as: hsumingwen1203 (901203). Use `wandb login --relogin` to force relogin


Step,Training Loss
500,0.462300
1000,0.386800
1500,0.372100
2000,0.370600
2500,0.276100
3000,0.285600
3500,0.220700
4000,0.213000
4500,0.174800


TrainOutput(global_step=4632, training_loss=0.30464462748051513, metrics={'train_runtime': 1442.924, 'train_samples_per_second': 25.673, 'train_steps_per_second': 3.21, 'total_flos': 3856159738466400.0, 'train_loss': 0.30464462748051513, 'epoch': 3.0})

In [9]:
trainer.save_model("./testmodel")

In [10]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis", model="./testmodel")


In [11]:
classifier(
    [
        "真的很棒",
        "只會來這一次了",
    ]
)

[{'label': 'LABEL_1', 'score': 0.9986364245414734},
 {'label': 'LABEL_0', 'score': 0.9826804995536804}]

In [12]:
model.config.id2label

{0: 'LABEL_0', 1: 'LABEL_1'}